This notebook demonstrates a few-shot prompting approach with the learnings made from previous attempts regarding Few-Shot prompting for LegalRuleML.

Future attempts should attempt to standardise the notation used. Llama's Grammar functionality could be of use.

In [ ]:
!pip install langchain langchain-openai pandas beautifulsoup4 requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.

In [ ]:
!pip install python-dotenv

In [ ]:
from secret_key import openai_key
import IK_interface

In [ ]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain_openai import OpenAI
import pandas as pd
import csv
from secret_key import openai_key

import IK_interface

In [ ]:
llm = OpenAI(openai_api_key=openai_key)


In [ ]:

examples = [
    {
        "question": "Statute: Punishment for piracy - Whoever commits any act of piracy shall be punished with imprisonment for life or with fine or with both; or with death if the act of piracy causes death.",
        "answer": """
<LegalRuleML xmlns="http://www.oasis-open.org/committees/legalruleml">
  <lrml:PrescriptiveStatement id="PunishmentForPiracy">
    <lrml:Rule id="RuleForPiracyPunishment">
      <lrml:if>
      <lrml:Fact>
          <lrml:Rel iri="#commitPiracy"/>
        </lrml:Fact>
      </lrml:if>
      <lrml:then>
        <lrml:Obligation>
          <lrml:Sanction>
            <lrml:Penalty>
              <lrml:Type iri="#imprisonment"/>
              <lrml:Duration max="life"/>
            </lrml:Penalty>
            <lrml:Condition>
              <lrml:CausesDeathOrAttempt/>
              <lrml:AdditionalSanctions>
                <lrml:Restitution/>
                <lrml:ForfeitureOfProperty/>
              </lrml:AdditionalSanctions>
            </lrml:Condition>
          </lrml:Sanction>
        </lrml:Obligation>
      </lrml:then>
    </lrml:Rule>
  </lrml:PrescriptiveStatement>
</LegalRuleML>"""}]

In [ ]:
example_prompt = PromptTemplate(
    input_variables=["question", "answer"], template="Question: {question}\n{answer}"
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)


In [ ]:
def law_parser():

    #IK_interface.extract_text(doc_number)
    csv_filename = 'fullsections.csv' # file containing all the sections from the E-Waste Rules, generated via IK_interface function


    # Read the CSV file into a Pandas DataFrame
    df = pd.read_csv(csv_filename)

    df2 = df.head(10)

    # Now, 'df' contains the data from the CSV file
    # You can perform various operations on the DataFrame
    combined_response = ''
    # Iterate through rows and process the data
    for index, row in df2.iterrows():
        section_title = row['Section Title']
        nested_content = row['Nested Content']

        # Process section_title and nested_content as needed
        #print(f"Section Title: {section_title}")
        #print(f"Nested Content: {nested_content}\n")
        combined_text = f"{section_title}: {nested_content}\n"

        prompt_for_input=prompt.format(input=combined_text)

        #print ("Input prompt is", prompt_for_input)

        response=llm.invoke(prompt_for_input)

        #print ("Response is", response)

        combined_response += response

    return combined_response

In [ ]:
combined_response=law_parser()

In [ ]:
print(combined_response)


<LegalRuleML xmlns="http://www.oasis-open.org/committees/legalruleml">
  <lrml:PrescriptiveStatement id="ShortTitleAndCommencement">
    <lrml:Rule id="RuleForShortTitle">
      <lrml:if>
        <lrml:Fact>
          <lrml:Rel iri="#hasTitle"/>
        </lrml:Fact>
      </lrml:if>
      <lrml:then>
        <lrml:Title>
          <lrml:Text>These rules may be called the E-Waste (Management) Rules, 2016.</lrml:Text>
        </lrml:Title>
      </lrml:then>
    </lrml:Rule>
    <lrml:Rule id="RuleForCommencement">
      <lrml:if>
        <lrml:Fact>
          <lrml:Rel iri="#hasCommencementDate"/>
        </lrml:Fact>
      </lrml:if>
      <lrml:then>
        <lrml:Commencement>
          <lrml:Date>1st day of October, 2016</lrml:Date>
        </lrml:
<LegalRuleML xmlns="http://www.oasis-open.org/committees/legalruleml">
  <lrml:PrescriptiveStatement id="ShortTitleAndCommencement">
    <lrml:Rule id="RuleForShortTitleAndCommencement">
      <lrml:then>
        <lrml:Obligation>
      